# FINAL

In [9]:
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objects as go
import pandas as pd
import pyodbc

# Conexión a la base de datos
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=localhost;'
    'DATABASE=datamart_proyecto15;'
    'UID=sa;'
    'PWD=1234567890'
)

# Función para obtener datos de las dimensiones independientes
def get_dimension_data():
    query_departments = "SELECT nombre_departamento AS Departamento FROM dimDepartamento"
    query_provinces = "SELECT nombre_provincia AS Provincia FROM dimProvincia"
    query_districts = "SELECT nombre_distrito AS Distrito FROM dimDistrito"
    query_time = "SELECT DISTINCT año AS Año, mes AS Mes FROM dimTiempo"

    df_departments = pd.read_sql(query_departments, conn)
    df_provinces = pd.read_sql(query_provinces, conn)
    df_districts = pd.read_sql(query_districts, conn)
    df_time = pd.read_sql(query_time, conn)

    return df_departments, df_provinces, df_districts, df_time

# Función para obtener datos relacionados al grado de instrucción
def get_instruction_data(selected_department=None, selected_province=None, selected_district=None, selected_year=None, selected_month=None):
    query = """
    SELECT 
        edu.gradoInstruccion,
        AVG(be.gastoEnProductos / NULLIF(be.gananciaNeta, 0)) AS PromedioRatioGastoIngreso,
        AVG(be.gananciaNeta - be.gastoEnProductos) AS PromedioDiferenciaIngresoGasto
    FROM factBalanceEconomico be
    JOIN dimEducacion edu ON be.idEducacion = edu.idEducacion
    JOIN dimDepartamento dep ON be.id_departamento = dep.id_departamento
    JOIN dimProvincia prov ON be.id_provincia = prov.id_provincia
    JOIN dimDistrito dist ON be.id_distrito = dist.id_distrito
    JOIN dimTiempo dt ON be.idTiempo = dt.idTiempo
    WHERE 1 = 1
    """

    if selected_department:
        query += f" AND dep.nombre_departamento = '{selected_department}'"
    if selected_province:
        query += f" AND prov.nombre_provincia = '{selected_province}'"
    if selected_district:
        query += f" AND dist.nombre_distrito = '{selected_district}'"
    if selected_year:
        query += f" AND dt.año = {selected_year}"
    if selected_month:
        query += f" AND dt.mes = {selected_month}"

    query += " GROUP BY edu.gradoInstruccion ORDER BY edu.gradoInstruccion"
    return pd.read_sql(query, conn)

# Función para obtener datos relacionados al género
def get_gender_data(selected_department=None, selected_province=None, selected_district=None, selected_year=None, selected_month=None):
    query = """
    SELECT 
        sx.sexo,
        AVG(be.gastoEnProductos / NULLIF(be.gananciaNeta, 0)) AS PromedioRatioGastoIngreso,
        AVG(be.gananciaNeta - be.gastoEnProductos) AS PromedioDiferenciaIngresoGasto
    FROM factBalanceEconomico be
    JOIN dimSexo sx ON be.idSexo = sx.idSexo
    WHERE 1 = 1
    """

    if selected_department:
        query += f" AND be.id_departamento IN (SELECT id_departamento FROM dimDepartamento WHERE nombre_departamento = '{selected_department}')"
    if selected_province:
        query += f" AND be.id_provincia IN (SELECT id_provincia FROM dimProvincia WHERE nombre_provincia = '{selected_province}')"
    if selected_district:
        query += f" AND be.id_distrito IN (SELECT id_distrito FROM dimDistrito WHERE nombre_distrito = '{selected_district}')"
    if selected_year:
        query += f" AND be.idTiempo IN (SELECT idTiempo FROM dimTiempo WHERE año = {selected_year})"
    if selected_month:
        query += f" AND be.idTiempo IN (SELECT idTiempo FROM dimTiempo WHERE mes = {selected_month})"

    query += " GROUP BY sx.sexo ORDER BY sx.sexo"
    return pd.read_sql(query, conn)

# Función para obtener datos de tipo de vivienda y estado
def get_housing_data(selected_department=None, selected_province=None, selected_district=None, selected_year=None, selected_month=None):
    query = """
    SELECT 
        tv.tipoVivienda AS TipoVivienda,
        ev.estadoVivienda AS EstadoVivienda,
        AVG(be.gastoEnProductos / NULLIF(be.gananciaNeta, 0)) AS PromedioRatioGastoIngreso
    FROM factBalanceEconomico be
    JOIN dimTipoVivienda tv ON be.idTipoVivienda = tv.idTipoVivienda
    JOIN dimEstadoVivienda ev ON be.idEstadoVivienda = ev.idEstadoVivienda
    WHERE 1 = 1
    """

    if selected_department:
        query += f" AND be.id_departamento IN (SELECT id_departamento FROM dimDepartamento WHERE nombre_departamento = '{selected_department}')"
    if selected_province:
        query += f" AND be.id_provincia IN (SELECT id_provincia FROM dimProvincia WHERE nombre_provincia = '{selected_province}')"
    if selected_district:
        query += f" AND be.id_distrito IN (SELECT id_distrito FROM dimDistrito WHERE nombre_distrito = '{selected_district}')"
    if selected_year:
        query += f" AND be.idTiempo IN (SELECT idTiempo FROM dimTiempo WHERE año = {selected_year})"
    if selected_month:
        query += f" AND be.idTiempo IN (SELECT idTiempo FROM dimTiempo WHERE mes = {selected_month})"

    query += """
    GROUP BY tv.tipoVivienda, ev.estadoVivienda
    ORDER BY tv.tipoVivienda, ev.estadoVivienda
    """
    return pd.read_sql(query, conn)

# Función para obtener datos regionales
def get_regional_data(selected_department=None, selected_province=None, selected_district=None, selected_year=None, selected_month=None):
    query = """
    SELECT 
        dep.nombre_departamento AS Departamento,
        AVG(be.gastoEnProductos / NULLIF(be.gananciaNeta, 0)) AS PromedioRatioGastoIngreso
    FROM factBalanceEconomico be
    JOIN dimDepartamento dep ON be.id_departamento = dep.id_departamento
    WHERE 1 = 1
    """

    if selected_department:
        query += f" AND dep.nombre_departamento = '{selected_department}'"
    if selected_province:
        query += f" AND be.id_provincia IN (SELECT id_provincia FROM dimProvincia WHERE nombre_provincia = '{selected_province}')"
    if selected_district:
        query += f" AND be.id_distrito IN (SELECT id_distrito FROM dimDistrito WHERE nombre_distrito = '{selected_district}')"
    if selected_year:
        query += f" AND be.idTiempo IN (SELECT idTiempo FROM dimTiempo WHERE año = {selected_year})"
    if selected_month:
        query += f" AND be.idTiempo IN (SELECT idTiempo FROM dimTiempo WHERE mes = {selected_month})"

    query += """
    GROUP BY dep.nombre_departamento
    ORDER BY dep.nombre_departamento
    """
    return pd.read_sql(query, conn)

# Función para obtener datos por año
def get_yearly_data(selected_department=None, selected_province=None, selected_district=None, selected_year=None, selected_month=None):
    query = """
    SELECT 
        dt.año AS Año,
        AVG(be.gastoEnProductos / NULLIF(be.gananciaNeta, 0)) AS PromedioRatioGastoIngreso,
        AVG(be.gananciaNeta - be.gastoEnProductos) AS PromedioDiferenciaIngresoGasto
    FROM factBalanceEconomico be
    JOIN dimTiempo dt ON be.idTiempo = dt.idTiempo
    WHERE 1 = 1
    """

    if selected_department:
        query += f" AND be.id_departamento IN (SELECT id_departamento FROM dimDepartamento WHERE nombre_departamento = '{selected_department}')"
    if selected_province:
        query += f" AND be.id_provincia IN (SELECT id_provincia FROM dimProvincia WHERE nombre_provincia = '{selected_province}')"
    if selected_district:
        query += f" AND be.id_distrito IN (SELECT id_distrito FROM dimDistrito WHERE nombre_distrito = '{selected_district}')"
    if selected_year:
        query += f" AND dt.año = {selected_year}"
    if selected_month:
        query += f" AND dt.mes = {selected_month}"

    query += """
    GROUP BY dt.año
    ORDER BY dt.año
    """
    return pd.read_sql(query, conn)

# Función para obtener datos laborales
def get_labor_data(selected_department=None, selected_province=None, selected_district=None, selected_year=None, selected_month=None):
    query = """
    SELECT 
        t.laborDesempeñada AS Labor,
        AVG(be.gastoEnProductos / NULLIF(be.gananciaNeta, 0)) AS PromedioRatioGastoIngreso
    FROM factBalanceEconomico be
    JOIN dimTrabajo t ON be.idTrabajo = t.idTrabajo
    WHERE 1 = 1
    """

    if selected_department:
        query += f" AND be.id_departamento IN (SELECT id_departamento FROM dimDepartamento WHERE nombre_departamento = '{selected_department}')"
    if selected_province:
        query += f" AND be.id_provincia IN (SELECT id_provincia FROM dimProvincia WHERE nombre_provincia = '{selected_province}')"
    if selected_district:
        query += f" AND be.id_distrito IN (SELECT id_distrito FROM dimDistrito WHERE nombre_distrito = '{selected_district}')"
    if selected_year:
        query += f" AND be.idTiempo IN (SELECT idTiempo FROM dimTiempo WHERE año = {selected_year})"
    if selected_month:
        query += f" AND be.idTiempo IN (SELECT idTiempo FROM dimTiempo WHERE mes = {selected_month})"

    query += """
    GROUP BY t.laborDesempeñada
    ORDER BY t.laborDesempeñada
    """
    return pd.read_sql(query, conn)

# Inicializar la app Dash
app = dash.Dash(__name__)
app.title = "Dashboard de Clase Media"

# Obtener datos para filtros
df_departments, df_provinces, df_districts, df_time = get_dimension_data()

# Diseño del dashboard
app.layout = html.Div([
    html.H1('Dashboard de Clase Media: Filtros y Gráficos', style={'textAlign': 'center', 'color': '#34495e', 'marginBottom': '20px'}),

    # Filtros en una cuadrícula 2x4
    html.Div([
        # Fila 1 de filtros
        html.Div([
            html.Div([
                html.Label('Departamento:', style={'fontWeight': 'bold'}),
                dcc.Dropdown(
                    id='department-filter',
                    options=[{'label': dept, 'value': dept} for dept in sorted(df_departments['Departamento'].unique())],
                    placeholder='Seleccione un departamento',
                    style={'backgroundColor': '#ecf0f1'}
                )
            ], style={'width': '23%', 'display': 'inline-block', 'paddingRight': '1%', 'marginBottom': '10px'}),

            html.Div([
                html.Label('Provincia:', style={'fontWeight': 'bold'}),
                dcc.Dropdown(
                    id='province-filter',
                    options=[{'label': prov, 'value': prov} for prov in sorted(df_provinces['Provincia'].unique())],
                    placeholder='Seleccione una provincia',
                    style={'backgroundColor': '#ecf0f1'}
                )
            ], style={'width': '23%', 'display': 'inline-block', 'paddingRight': '1%', 'marginBottom': '10px'}),

            html.Div([
                html.Label('Distrito:', style={'fontWeight': 'bold'}),
                dcc.Dropdown(
                    id='district-filter',
                    options=[{'label': dist, 'value': dist} for dist in sorted(df_districts['Distrito'].unique())],
                    placeholder='Seleccione un distrito',
                    style={'backgroundColor': '#ecf0f1'}
                )
            ], style={'width': '23%', 'display': 'inline-block', 'paddingRight': '1%', 'marginBottom': '10px'}),

            html.Div([
                html.Label('Año:', style={'fontWeight': 'bold'}),
                dcc.Dropdown(
                    id='year-filter',
                    options=[{'label': year, 'value': year} for year in sorted(df_time['Año'].unique())],
                    placeholder='Seleccione un año',
                    style={'backgroundColor': '#ecf0f1'}
                )
            ], style={'width': '23%', 'display': 'inline-block', 'marginBottom': '10px'})
        ], style={'width': '100%', 'display': 'flex', 'justifyContent': 'space-between'}),

        # Fila 2 de filtros
        html.Div([
            html.Div([
                html.Label('Mes:', style={'fontWeight': 'bold'}),
                dcc.Dropdown(
                    id='month-filter',
                    options=[{'label': month, 'value': month} for month in sorted(df_time['Mes'].unique())],
                    placeholder='Seleccione un mes',
                    style={'backgroundColor': '#ecf0f1'}
                )
            ], style={'width': '23%', 'display': 'inline-block', 'paddingRight': '1%', 'marginBottom': '10px'}),

            # Espacios vacíos para completar la cuadrícula 2x4
            html.Div([], style={'width': '23%', 'display': 'inline-block', 'paddingRight': '1%', 'marginBottom': '10px'}),
            html.Div([], style={'width': '23%', 'display': 'inline-block', 'paddingRight': '1%', 'marginBottom': '10px'}),
            html.Div([], style={'width': '23%', 'display': 'inline-block', 'marginBottom': '10px'})
        ], style={'width': '100%', 'display': 'flex', 'justifyContent': 'space-between'})
    ], style={'width': '90%', 'margin': 'auto', 'backgroundColor': '#bdc3c7', 'padding': '20px', 'borderRadius': '10px', 'boxShadow': '0 4px 8px rgba(0, 0, 0, 0.1)'}),

    html.Br(),

    # Gráficos organizados en una cuadrícula con dos gráficos por fila
    html.Div([
        # Primera fila de gráficos
        html.Div([
            html.Div([
                dcc.Graph(id='ratio-chart')
            ], style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top', 'padding': '1%'}),

            html.Div([
                dcc.Graph(id='difference-chart')
            ], style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top', 'padding': '1%'})
        ], style={'width': '100%', 'display': 'flex', 'justifyContent': 'space-between'}),

        # Segunda fila de gráficos
        html.Div([
            html.Div([
                dcc.Graph(id='housing-chart')
            ], style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top', 'padding': '1%'}),

            html.Div([
                dcc.Graph(id='gender-combined-chart')
            ], style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top', 'padding': '1%'})
        ], style={'width': '100%', 'display': 'flex', 'justifyContent': 'space-between'}),

        # Tercera fila de gráficos
        html.Div([
            html.Div([
                dcc.Graph(id='regional-chart')
            ], style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top', 'padding': '1%'}),

            html.Div([
                dcc.Graph(id='yearly-chart')
            ], style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top', 'padding': '1%'})
        ], style={'width': '100%', 'display': 'flex', 'justifyContent': 'space-between'}),

        # Cuarta fila de gráficos (solo un gráfico)
        html.Div([
            html.Div([
                dcc.Graph(id='labor-chart')
            ], style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top', 'padding': '1%'})
        ], style={'width': '100%', 'display': 'flex', 'justifyContent': 'flex-start'})
    ], style={'width': '90%', 'margin': 'auto'})
], style={'backgroundColor': '#f0f3f5', 'padding': '20px'})

# Callback para actualizar los gráficos
@app.callback(
    [
        Output('ratio-chart', 'figure'),
        Output('difference-chart', 'figure'),
        Output('housing-chart', 'figure'),
        Output('gender-combined-chart', 'figure'),
        Output('regional-chart', 'figure'),
        Output('yearly-chart', 'figure'),
        Output('labor-chart', 'figure')
    ],
    [
        Input('department-filter', 'value'),
        Input('province-filter', 'value'),
        Input('district-filter', 'value'),
        Input('year-filter', 'value'),
        Input('month-filter', 'value')
    ]
)
def update_graphs(selected_department, selected_province, selected_district, selected_year, selected_month):
    # Obtener datos filtrados
    df_instruction = get_instruction_data(selected_department, selected_province, selected_district, selected_year, selected_month)
    df_gender = get_gender_data(selected_department, selected_province, selected_district, selected_year, selected_month)
    df_housing = get_housing_data(selected_department, selected_province, selected_district, selected_year, selected_month)
    df_regional = get_regional_data(selected_department, selected_province, selected_district, selected_year, selected_month)
    df_yearly = get_yearly_data(selected_department, selected_province, selected_district, selected_year, selected_month)
    df_labor = get_labor_data(selected_department, selected_province, selected_district, selected_year, selected_month)

    # Gráfico 1: Promedio Ratio Gasto/Ingreso por Grado de Instrucción
    ratio_fig = go.Figure()
    ratio_fig.add_trace(go.Bar(
        x=df_instruction['gradoInstruccion'],
        y=df_instruction['PromedioRatioGastoIngreso'],
        marker_color='#3498db'
    ))
    ratio_fig.update_layout(
        title='Promedio Ratio Gasto/Ingreso por Grado de Instrucción',
        xaxis_title='Grado de Instrucción',
        yaxis_title='Promedio Ratio Gasto/Ingreso',
        plot_bgcolor='white',
        paper_bgcolor='white',
        margin=dict(l=40, r=40, t=40, b=40)
    )

    # Gráfico 2: Promedio Diferencia Ingreso-Gasto por Grado de Instrucción
    difference_fig = go.Figure()
    difference_fig.add_trace(go.Bar(
        x=df_instruction['gradoInstruccion'],
        y=df_instruction['PromedioDiferenciaIngresoGasto'],
        marker_color='#2ecc71'
    ))
    difference_fig.update_layout(
        title='Promedio Diferencia Ingreso-Gasto por Grado de Instrucción',
        xaxis_title='Grado de Instrucción',
        yaxis_title='Promedio Diferencia Ingreso-Gasto',
        plot_bgcolor='white',
        paper_bgcolor='white',
        margin=dict(l=40, r=40, t=40, b=40)
    )

    # Gráfico 3: Promedio Ratio Gasto/Ingreso por Tipo de Vivienda y Estado (Barras Apiladas)
    housing_fig = go.Figure()
    for estado in df_housing['EstadoVivienda'].unique():
        estado_data = df_housing[df_housing['EstadoVivienda'] == estado]
        housing_fig.add_trace(go.Bar(
            x=estado_data['TipoVivienda'],
            y=estado_data['PromedioRatioGastoIngreso'],
            name=estado
        ))
    housing_fig.update_layout(
        title='Promedio Ratio Gasto/Ingreso por Tipo de Vivienda y Estado',
        xaxis_title='Tipo de Vivienda',
        yaxis_title='Promedio Ratio Gasto/Ingreso',
        barmode='stack',
        plot_bgcolor='white',
        paper_bgcolor='white',
        margin=dict(l=40, r=40, t=40, b=40),
        legend_title_text='Estado de Vivienda'
    )

    # Gráfico 4: Promedio Ratio Gasto/Ingreso y Diferencia Ingreso-Gasto por Género
    gender_combined_fig = go.Figure()
    gender_combined_fig.add_trace(go.Bar(
        x=df_gender['sexo'],
        y=df_gender['PromedioRatioGastoIngreso'],
        name='Promedio Ratio Gasto/Ingreso',
        marker_color='#e74c3c'
    ))
    gender_combined_fig.add_trace(go.Bar(
        x=df_gender['sexo'],
        y=df_gender['PromedioDiferenciaIngresoGasto'],
        name='Promedio Diferencia Ingreso-Gasto',
        marker_color='#f1c40f'
    ))
    gender_combined_fig.update_layout(
        title='Promedio Ratio Gasto/Ingreso y Diferencia Ingreso-Gasto por Género',
        xaxis_title='Género',
        yaxis_title='Promedio',
        barmode='group',
        plot_bgcolor='white',
        paper_bgcolor='white',
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=0.9,
            xanchor='center',
            x=0.5
        ),
        margin=dict(l=40, r=40, t=40, b=40)
    )

    # Gráfico 5: Promedio Ratio Gasto/Ingreso por Departamento
    regional_fig = go.Figure()
    regional_fig.add_trace(go.Bar(
        x=df_regional['Departamento'],
        y=df_regional['PromedioRatioGastoIngreso'],
        marker_color='#9b59b6'
    ))
    regional_fig.update_layout(
        title='Promedio Ratio Gasto/Ingreso por Departamento',
        xaxis_title='Departamento',
        yaxis_title='Promedio Ratio Gasto/Ingreso',
        plot_bgcolor='white',
        paper_bgcolor='white',
        margin=dict(l=40, r=40, t=40, b=40)
    )

    # Gráfico 6: Evolución del Ratio Gasto/Ingreso y Diferencia Ingreso-Gasto por Año
    yearly_fig = go.Figure()
    yearly_fig.add_trace(go.Scatter(
        x=df_yearly['Año'],
        y=df_yearly['PromedioRatioGastoIngreso'],
        mode='lines+markers',
        name='Promedio Ratio Gasto/Ingreso',
        line=dict(color='#1abc9c')
    ))
    yearly_fig.add_trace(go.Scatter(
        x=df_yearly['Año'],
        y=df_yearly['PromedioDiferenciaIngresoGasto'],
        mode='lines+markers',
        name='Promedio Diferencia Ingreso-Gasto',
        line=dict(color='#e67e22')
    ))
    yearly_fig.update_layout(
        title='Evolución del Ratio Gasto/Ingreso y Diferencia Ingreso-Gasto por Año',
        xaxis_title='Año',
        yaxis_title='Promedio',
        template='plotly_white',
        plot_bgcolor='white',
        paper_bgcolor='white',
        margin=dict(l=40, r=40, t=40, b=40),
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=0.9,
            xanchor='center',
            x=0.5
        )
    )

    # Gráfico 7: Promedio Ratio Gasto/Ingreso por Labor Desempeñada
    labor_fig = go.Figure()
    labor_fig.add_trace(go.Bar(
        x=df_labor['Labor'],
        y=df_labor['PromedioRatioGastoIngreso'],
        marker_color='#34495e'
    ))
    labor_fig.update_layout(
        title='Promedio Ratio Gasto/Ingreso por Labor Desempeñada',
        xaxis_title='Labor Desempeñada',
        yaxis_title='Promedio Ratio Gasto/Ingreso',
        plot_bgcolor='white',
        paper_bgcolor='white',
        margin=dict(l=40, r=40, t=40, b=40)
    )

    return ratio_fig, difference_fig, housing_fig, gender_combined_fig, regional_fig, yearly_fig, labor_fig

if __name__ == '__main__':
    app.run_server(debug=True)


C:\Users\claud\AppData\Local\Temp\ipykernel_15496\2186434514.py:23: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

C:\Users\claud\AppData\Local\Temp\ipykernel_15496\2186434514.py:24: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

C:\Users\claud\AppData\Local\Temp\ipykernel_15496\2186434514.py:25: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

C:\Users\claud\AppData\Local\Temp\ipykernel_15496\2186434514.py:26: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 

C:\Users\claud\AppData\Local\Temp\ipykernel_15496\2186434514.py:58: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

C:\Users\claud\AppData\Local\Temp\ipykernel_15496\2186434514.py:84: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

C:\Users\claud\AppData\Local\Temp\ipykernel_15496\2186434514.py:114: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

C:\Users\claud\AppData\Local\Temp\ipykernel_15496\2186434514.py:142: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI